# Grid World Tutorial: POMDPs.jl for Complete Beginners

In this tutorial, we try to provide a simple example of how to define a Markov decision process (MDP) problem using the [POMDPs.jl](https://github.com/sisl/POMDPs.jl) interface. After defining the problem in this way, you will be able to use the solvers that the interface supports. In this tutorial, we will show you how to use the value iteration and the Monte Carlo Tree Search solvers that the POMDPs.jl interface supports. We assume that you have some knowledge of basic programming, but are not necessarily familiar with all the features that exist in Julia. We try to cover the many language specific features used in POMDPs.jl in this tutorial. We do assume that you know the grid world problem, and are familiar with the formal defintion of the MDP.  Let's get started!

## Dependencies
You need to install a few modules in order to use this notebook. If you have all the modules below installed, great! If not run the following commands:
```julia
# install the POMDPs.jl interface
Pkg.clone("https://github.com/sisl/POMDPs.jl.git")

# install the Value Iteration solver
Pkg.clone("https://github.com/sisl/DiscreteValueIteration.jl")

# install the MCTS solver
Pkg.clone("https://github.com/sisl/MCTS.jl")

# install a helper module for handling distriubtions
Pkg.clone("https://github.com/sisl/POMDPDistributions.jl")
```

In [1]:
# first import the POMDPs.jl interface
using POMDPs
# we also use a helper module to work with probability distributions
using POMDPDistributions

## Problem Overview
In Grid World, we are trying to control an agent who has trouble moving in the desired direction. In our problem, we have a four reward states on a $10\times 10$ grid. Each position on the grid represents a state, and the positive reward states are terminal (the agent stops recieveing reward after reaching them). The agent has four actions to choose from: up, down, left, right. The agent moves in the desired direction with a probability of 0.7, and with a probability of 0.1 in each of the remaining three directions. The problem has the following form:
![example](http://artint.info/figures/ch09/gridworldc.gif)

## States
The data container below represents the state of the agent in the grid world. Note the 
```julia
type GridWorldState <: State
``` 
syntax. GridWorldState is inheriting the behavior of the abstract State type (defined in POMDPs.jl). It simply means that GridWorldState is a child of State. If you are interested in in learning more about the type system and inheritance in Jullia check out [this](http://chrisvoncsefalvay.com/2015/02/04/types-and-inheritance-in-julia/) blog post. If you are confused by the syntax, don't worry. All you have to remeber is to add ```<: State``` in the dfinition of the state type for your problem. 

In [2]:
type GridWorldState <: State
    x::Int64 # x position
    y::Int64 # y position
    bumped::Bool # did we bump the wall?
    done::Bool # are we in a terminal state?
end

Below are some convenience functions for working with the GridWorldState. 

In [3]:
# initial state constructor
GridWorldState(x::Int64, y::Int64) = GridWorldState(x,y,false,false)
# checks if the position of two states are the same
posequal(s1::GridWorldState, s2::GridWorldState) = s1.x == s2.x && s1.y == s2.y
# copies state s2 to s2
function Base.copy!(s1::GridWorldState, s2::GridWorldState) 
    s1.x = s2.x
    s1.y = s2.y
    s1.bumped = s2.bumped
    s1.done = s2.done
    s1
end
# if you want to use Monte Carlo Tree Search, you will need to define the functions below
Base.hash(s::GridWorldState, h::UInt64 = zero(UInt64)) = hash(s.x, hash(s.y, hash(s.bumped, hash(s.done, h))))
Base.isequal(s1::GridWorldState,s2::GridWorldState) = s1.x == s2.x && s1.y == s2.y && s1.bumped == s2.bumped && s1.done == s2.done;

## Actions
The data container below holds the agent's action. Since our action is simply the direction the agent chooses to go (i.e. up, down, left, right), we can use a Symbol to represent it. Symbols are special types in Julia that allow for nice represntation of complex data. However, in our case a string could serve the same purpose as the symbol, so feel free to use what you're most comfortable with. Here we use the <: notation to indicate the GridWorldAction is a subtype of Action.

In [4]:
type GridWorldAction <: Action
    direction::Symbol
end

## MDP
The GridWorld data container is defined below. It holds all the information we need to define the MDP tuple $$(\mathcal{S}, \mathcal{A}, T, R).$$

In [5]:
# the grid world mdp type
type GridWorld <: POMDP
    size_x::Int64 # x size of the grid
    size_y::Int64 # y size of the grid
    reward_states::Vector{GridWorldState} # the states in which agent recieves reward
    reward_values::Vector{Float64} # reward values for those states
    bounds_penalty::Float64 # penalty for bumping the wall
    tprob::Float64 # probability of transitioning to the desired state
    discount_factor::Float64 # disocunt factor
end

Dont't be confused by the ```<: POMDP```. Although as defined here, GridWorld is not a POMDP, the POMDPs.jl interface and solvers support both MDPs and POMDPs under one common abstract type POMDP. 

Before moving on, I want to create a constructor for GridWorld for convenience. Currently, if I want to create an instance of GridWorld, I have to pass in all of fields inside the GridWorld container (size_x, size_y, etc). The function below will return a GridWorld type with all the fields filled with some default values.

In [6]:
# we use key worded arguments so we can change any of the values we pass in 
function GridWorld(;sx::Int64=10, # size_x
                    sy::Int64=10, # size_y
                    rs::Vector{GridWorldState}=[GridWorldState(4,3), GridWorldState(4,6), GridWorldState(9,3), GridWorldState(8,8)], # reward states
                    rv::Vector{Float64}=rv = [-10.,-5,10,3], # reward values
                    penalty::Float64=-1.0, # bounds penalty
                    tp::Float64=0.7, # tprob
                    discount_factor::Float64=0.9)
    return GridWorld(sx, sy, rs, rv, penalty, tp, discount_factor)
end

# we can now create a GridWorld mdp instance like this:
mdp = GridWorld()
mdp.reward_states # mdp contains all the defualt values from the constructor

4-element Array{GridWorldState,1}:
 GridWorldState(4,3,false,false)
 GridWorldState(4,6,false,false)
 GridWorldState(9,3,false,false)
 GridWorldState(8,8,false,false)

## Spaces
Let's look at how we can define the state and action spaces for our problem.

### State Space
The state space in an MDP represents all the states in the problem. There are two primary functionalities that we want our spaces to support. We want to be able to iterate over the state space (for Value Iteration for example), and sometimes we want to be able to sample form the state space (used in some POMDP solvers). In this notebook, we will only look at iterable state spaces. 

In [7]:
type StateSpace <: AbstractSpace
    states::Vector{GridWorldState}
end

Since we can iterate over elements of an array, and our problem is small, we can store all of our states in an array. If your problem is very large (tens of millions of states), it might be worthwhile to create an iterator over your state space. See [this](http://stackoverflow.com/questions/25028539/how-to-implement-an-iterator-in-julia) post on stackoverflow on making simple iterators. 

In [8]:
function POMDPs.states(mdp::GridWorld)
    s = GridWorldState[] # initialize an array of GridWorldStates
    # loop over all our states, remeber there are two binary variables:
    # done (d) and bumped(b)
    for d = 0:1, b = 0:1, y = 1:mdp.size_y, x = 1:mdp.size_x
        push!(s, GridWorldState(x,y,b,d))
    end
    return StateSpace(s)
end;

Here, the code: ```function POMDPs.states(mdp::GridWorld)``` means that we want to take the function called ```states(...)``` from the POMDPs.jl module and add another method to it. The ```states(...)``` method in POMDPs.jl doesn't know about our GridWorld type. However, now when ```states(...)``` is called with GridWorld it will dispatch the function we defined above! This is the awesome thing about multiple-dispatch, and one of the features that should make working with MDP/POMDPs easier in Julia. 

The solvers that suppor the POMDPs.jl interface know that a function called ```states(...)``` exists in the interface. However, they do not know the behavior of that function for GridWorld. That means in order for the solvers to use this behavior all we have to do is pass an instance of our GridWorld type into the solver. When ```states(...)``` is called in the solver with the GridWorld type, the function above will be called.   

In [9]:
# let's use the constructor for GridWorld we defined earlier
mdp = GridWorld()
state_space = states(mdp);
state_space.states[1] # remeber that our state space instance has an array called states in it

GridWorldState(1,1,false,false)

In Julia, a method may only work on a specific type. It is up to the program writer to specify what type a given function will work on. This might seem tedious at first, but the Julia type system adds a lot of flexibility to large software frameworks, and allows us to easily extend functionality of any type. 

So what happens if we pass something else to ```states(...)```? 

In [10]:
states("hi") # this is the default error julia throws

LoadError: LoadError: MethodError: `states` has no method matching states(::ASCIIString)
while loading In[10], in expression starting on line 1

What if we pass in a type that inherits from the POMDP abstract type.

In [11]:
type TestMDP <: POMDP
    x::Int64
end
tmdp = TestMDP(1)
states(tmdp) # this throws a more relevant error

LoadError: LoadError: POMDPs.jl: No implementation of states for pomdp::TestMDP.
while loading In[11], in expression starting on line 5

The interface in POMDPs.jl defines the function names (API) that the supported MDP/POMDP solvers are allowed to call. In Julia, the functions do not belong to a specific type, but  are dispatched based on the type passed in (i.e. functions are generic). Once we define the ```states(mdp::TestMDP)``` function, the solvers will be able to call it. If you forget to define a function that a solver requires, an error like the one above will be thrown. 

To finish up we need to define a function that returns the iterator over the state space. Remeber ```states(...)``` returns the state space type, and our iterator is hidden inside of it.

In [12]:
function POMDPs.iterator(space::StateSpace)
    return space.states 
end;

In value iteration, for example, the solver will iterrate over your state space by doing the following:

In [13]:
mdp = GridWorld()
state_space = states(mdp);
for s in iterator(state_space)
    # value iteration applies the bellman operator to your state s
end

We want to be able to uniformly sample for our state space. A sampling function for doing this is fairly simple to implement if you already have an array of all your states.

In [14]:
function POMDPs.rand(rng::AbstractRNG, space::StateSpace, s::GridWorldState)
    sp = space.states[rand(rng, 1:end)]
    copy!(s, sp)
    s
end;

### Action Space

The action space is the set of all actions availiable to the agent. In the grid world problem the action space consists of up, down, left, and right. Let's define the type for our action space.

In [15]:
type ActionSpace <: AbstractSpace
    actions::Vector{GridWorldAction}
end

Let's now write a function called actions that returns our action space.

In [16]:
function POMDPs.actions(mdp::GridWorld)
    acts = [GridWorldAction(:up), GridWorldAction(:down),
    GridWorldAction(:left), GridWorldAction(:right)]
    return ActionSpace(acts)
end;
POMDPs.actions(mdp::GridWorld, s::GridWorldState, as::ActionSpace=actions(mdp)) = as;

Finally, let's make the iterator function for our action space.

In [17]:
function POMDPs.iterator(space::ActionSpace)
    return space.actions 
end;

The solvers will iterrate over the action space in a similar way.

Let's add a function to sample from the action space.

In [18]:
function POMDPs.rand(rng::AbstractRNG, space::ActionSpace, a::GridWorldAction)
    ap = space.actions[rand(rng, 1:end)]
    a.direction = ap.direction
    a
end;
function POMDPs.rand(rng::AbstractRNG, space::ActionSpace)
    a = GridWorldAction(:left)
    return rand(rng, space, a)
end;

To finish up, we add a couple of initializer functions for out state and action.

In [19]:
POMDPs.create_state(mdp::GridWorld) = GridWorldState(1,1)
POMDPs.create_action(mdp::GridWorld) = GridWorldAction(:up);

Now that we've defined our state and action spaces, we are half-way thorugh our MDP tuple:
$$
(\mathcal{S}, \mathcal{A}, T, R)
$$

## Distributions

Since MDPs are probabilistic models, we have to deal with probability distributions. In this section, we outline how to define probability distriubtions, and what tools are availiable to help you with the task.

### Transition Distribution 

If you are familiar with MDPs, you know that the transition function $T(s' \mid s, a)$ captures the dynamics of the system. Specifically, $T(s' \mid s, a)$ is a real value that defines the probabiltiy of transitioning to state $s'$ given that you took action $a$ in state $s$. The transition distirubtion $T(\cdot \mid s, a)$ is a slightly different construct. This is the actual distribution over the states that our agent can reach given that its in state $s$ and took action $a$. In other words this is the distribution over $s'$. 

There are many ways to implement transition distributions for your problem. Your choice of distribution as well as how you implement it will heavily depend on your problem. [Distributions.jl](https://github.com/JuliaStats/Distributions.jl) provides support for many common univariate and multivarite distributions. Below is how we implement the one for grid world. 

In [20]:
type GridWorldDistribution <: AbstractDistribution
    neighbors::Array{GridWorldState} # the states s' in the distribution
    probs::Array{Float64} # the probability corresponding to each state s'
    cat::Categorical # this comes from POMDPDistributions.jl and is used for sampling
end

To reduce memory allocation, the POMDPs.jl interface defines some initalization functions that return initial types to be filled lated. This function returns the distribution type filled with some values. We don't care what the distribution container has in it, because it will be modified at each call to the transition function.

In [21]:
function POMDPs.create_transition_distribution(mdp::GridWorld)
    # can have at most five neighbors in grid world
    neighbors =  [GridWorldState(i,i) for i = 1:5]
    probabilities = zeros(5) + 1.0/5.0
    cat = Categorical(5)
    return GridWorldDistribution(neighbors, probabilities, cat)
end;

Notice that there are a maximum of five neighboring states $s'$ that can be reached from a given state action pair. There is a neighboring grid space in each direction around us (4 in total), and the grid space we are currently in for a total of five.

The next function we want is ```domain(...)```. For discrete state distributions, domain returns an iterator over the states in that distributions (this is just the neighbors array in our distriubtion type). The function takes on the following form:

In [22]:
function POMDPs.iterator(d::GridWorldDistribution)
    return d.neighbors
end;

Let's implement the probability density function (really this is a probability mass function since the distriubtion is discrete, but we overload the pdf function name to serve as both). Below is a fairly inneficient impelemntation of pdf. For the discrete distribution in our problem, the pdf function returns the probability of drawing the state s from the distribution d. 

In [23]:
function POMDPs.pdf(d::GridWorldDistribution, s::GridWorldState)
    for (i, sp) in enumerate(d.neighbors)
        if s == sp
            return d.probs[i]
        end
    end   
    return 0.0
end;

Finally, we want to implement a smapling function that can draw samples from our distribution. Once again, there are many ways to do this, but we recommend using Distributions.jl. We use POMDPDistributions which mimicks a lot of the behavior of Distributions.jl. 

In [24]:
function POMDPs.rand(rng::AbstractRNG, d::GridWorldDistribution, s::GridWorldState)
    set_prob!(d.cat, d.probs) # fill the Categorical distribution with our state probabilities
    ns = d.neighbors[rand(rng, d.cat)] # sample a neighbor state according to the distribution c
    copy!(s, ns)
    return s # return the pointer to s
end;

There are a few things that might look scary in the rand! function. Don't worry they aren't!

First the ! notation. By Julia convention, when we write a function that changes its imput arguments, we end that function name with a !. This signature tells anyone calling the function that their input arguments will change. In our function we sample a state from our distribution d, and change s to be that state.

Second, one of the arguments into rand! is AbstractRNG. What in the world is that? This is a special type in Julia that sets the internal seed of the random number generator. Let's take a look at an example.

Below we initialize a grid world MDP, a transition distribution and a state. We also use the MeresenneTwister type as our AbstractRNG. MeresenneTwister is a Julia type used for pseudo random number generation. 

In [25]:
mdp = GridWorld()
# the function below initializes our distriubtion d to have the states at:
# (1,1) (2,2) (3,3) (4,4) (5,5)
# we should expect to sample only these states from d
d = create_transition_distribution(mdp) 
s = GridWorldState(1,1)
rng = MersenneTwister(1) # this is an rng type in Julia

for i = 1:5
    rand(rng, d, s)
    println(s)
end

GridWorldState(2,2,false,false)
GridWorldState(2,2,false,false)
GridWorldState(2,2,false,false)
GridWorldState(1,1,false,false)
GridWorldState(3,3,false,false)


Now if we seed the MersenneTwister with the same number again, we will sample the same sequence of staes from the distribution d.

In [26]:
rng = MersenneTwister(1) 

for i = 1:5
    rand(rng, d, s)
    println(s)
end

GridWorldState(2,2,false,false)
GridWorldState(2,2,false,false)
GridWorldState(2,2,false,false)
GridWorldState(1,1,false,false)
GridWorldState(3,3,false,false)


If we set the MersenneTwister seed to something else, we will get a different sequence of states. 

In [27]:
rng = MersenneTwister(2) 

for i = 1:5
    rand(rng, d, s)
    println(s)
end

GridWorldState(2,2,false,false)
GridWorldState(3,3,false,false)
GridWorldState(2,2,false,false)
GridWorldState(5,5,false,false)
GridWorldState(3,3,false,false)


Using an AbstractRNG is very similar to setting a random seed. This type of functionality is very useful when you are trying to generate the same sequence of states in a Monte Carlo simulation for example.

To recap, there are two functionalities that we require your distirbutions to support. We want to be able to sample from them using the ```rand!(...)``` function, and we want to obtain the probability density using the ```pdf(...)``` function.

## Transition Model

In this section we will define the system dynamics of the gird world MDP. In POMDPs.jl, we work with transition distirbution functions $T(s' \mid s, a)$, so we want to write a function that can generate the transition distributions over $s'$ for us given an $(s, a)$ pair. 

In grid world, the dynamics of the system are fairly simple. We move in the specified direction with some pre-defined probability. This is the tprob parameter in our GridWorld MDP (it is set to 0.7 in the DMU book example).  If we bump against a wall, we recieve a penalty. If we get to state with a positive reward, we've reached a terminal state and can no longer accumulate reward.

In the transition function we want to fill the neighbors in our distribution d with the reachable states from the state, action pair. We want to fill the probs in our distirbution d with the probabilities of reaching that neighbor. 

In [28]:
# transition helpers
function inbounds(mdp::GridWorld,x::Int64,y::Int64)
    if 1 <= x <= mdp.size_x && 1 <= y <= mdp.size_y
        return true
    else
        return false
    end
end

function inbounds(mdp::GridWorld,state::GridWorldState)
    x = state.x #point x of state
    y = state.y
    return inbounds(mdp, x, y)
end

function fill_probability!(p::Vector{Float64}, val::Float64, index::Int64)
    for i = 1:length(p)
        if i == index
            p[i] = val
        else
            p[i] = 0.0
        end
    end
end;


In [29]:
function POMDPs.transition(mdp::GridWorld,
                           state::GridWorldState,
                           action::GridWorldAction,
                           d::GridWorldDistribution=create_transition_distribution(mdp))
    # re-assign names so we don't have to call actiond.direction every time etc
    a = action.direction
    x = state.x
    y = state.y
    
    neighbors = d.neighbors
    probability = d.probs
    
    # let's handle the done case first
    if state.done
        # can only transition to the same done state
        fill!(probability, 0.0)
        probability[1] = 1.0
        copy!(neighbors[1], state)
        # when we sample d, we will only get the state in neighbors[1] - our done state
        return d
    end
    
    fill!(probability, 0.1)
    probability[5] = 0.0

    neighbors[1].x = x+1; neighbors[1].y = y
    neighbors[2].x = x-1; neighbors[2].y = y
    neighbors[3].x = x; neighbors[3].y = y-1
    neighbors[4].x = x; neighbors[4].y = y+1
    neighbors[5].x = x; neighbors[5].y = y
    
    for i = 1:5 neighbors[i].bumped = false end
    for i = 1:5 neighbors[i].done = false end
    reward_states = mdp.reward_states
    reward_values = mdp.reward_values
    n = length(reward_states)
    for i = 1:n
        #if state == reward_states[i] && reward_values[i] > 0.0
        if posequal(state, reward_states[i]) && reward_values[i] > 0.0
            fill_probability!(probability, 1.0, 5)
            neighbors[5].done = true
            return d
        end
    end

    if a == :right
        if !inbounds(mdp, neighbors[1])
            fill_probability!(probability, 1.0, 5)
            neighbors[5].bumped = true
        else
            probability[1] = 0.7
        end

    elseif a == :left
        if !inbounds(mdp, neighbors[2])
            fill_probability!(probability, 1.0, 5)
            neighbors[5].bumped = true
        else
            probability[2] = 0.7
        end

    elseif a == :down
        if !inbounds(mdp, neighbors[3])
            fill_probability!(probability, 1.0, 5)
            neighbors[5].bumped = true
        else
            probability[3] = 0.7
        end

    elseif a == :up
        if !inbounds(mdp, neighbors[4])
            fill_probability!(probability, 1.0, 5)
            neighbors[5].bumped = true
        else
            probability[4] = 0.7
        end
    end

    count = 0
    new_probability = 0.1

    for i = 1:length(neighbors)
        if !inbounds(mdp, neighbors[i])
         count += 1
            probability[i] = 0.0
         end
     end

    if count == 1
        new_probability = 0.15
    elseif count == 2
        new_probability = 0.3
    end

    if count > 0
        for i = 1:length(neighbors)
            if probability[i] == 0.1
               probability[i] = new_probability
            end
        end
    end
    d
end;

## Reward Model
The reward model $R(s,a,s')$ is a function that returns the reward of being in state $s$, taking an action $a$ from that state, and ending up in state $s'$. In our problem, we are rewarded for reaching a terimanl reward state (this could be positive or negative), and we are penalized for bumping into a wall.

In [30]:
function POMDPs.reward(mdp::GridWorld, state::GridWorldState, action::GridWorldAction, statep::GridWorldState) #deleted action
    if state.done
        return 0.0
    end
    r = 0.0
    reward_states = mdp.reward_states
    reward_values = mdp.reward_values
    n = length(reward_states)
    for i = 1:n
        if posequal(state, reward_states[i])
            r += reward_values[i]
        end
    end
    if state.bumped
        r += mdp.bounds_penalty
    end
    return r
end;


## Miscallenous Functions
We are almost done! Just a few simple functions left. First let's implement two functions that return the sizes of our state and action spaces.

In [31]:
POMDPs.n_states(mdp::GridWorld) = 4*mdp.size_x*mdp.size_y
POMDPs.n_actions(mdp::GridWorld) = 4;

Now, we implement the discount function.

In [32]:
POMDPs.discount(mdp::GridWorld) = mdp.discount_factor;

The last thing we need is an indexing function. This allows us to index between the discrete utility array and the states in our problem. We will use the ```sub2ind()``` function from Julia base to help us here. 

In [33]:
function POMDPs.index(mdp::GridWorld, state::GridWorldState)
    sb = Int(state.bumped + 1)
    sd = Int(state.done + 1)
    return sub2ind((mdp.size_x, mdp.size_y, 2, 2), state.x, state.y, sb, sd)
end;

Finally let's define a function that checks if a state is terminal.

In [34]:
function POMDPs.isterminal(mdp::GridWorld, s::GridWorldState)
    s.done ? (return true) : (return false)
end;

## Value Iteration Solver

Value iteration is a dynamic porgramming apporach for solving MDPs. See the [wikipedia](https://en.wikipedia.org/wiki/Markov_decision_process#Value_iteration) article for a brief explanation. The solver can be found [here](https://github.com/sisl/DiscreteValueIteration.jl). If you haven't isntalled the solver yet, you can run the following from the Julia REPL
```julia
Pkg.clone("https://github.com/sisl/DiscreteValueIteration.jl")
```
to download the module.

Each POMDPs.jl solver provides two data types for you to interface with. The first is the Solver type which contains solver parameters. The second is the Policy type. Let's see hwo we can use them to get an optimal action at a given state.

In [35]:
# first let's load the value iteration module
using DiscreteValueIteration

# initialize the problem
mdp = GridWorld()

# initialize the solver
# max_iterations: maximum number of iterations value iteration runs for (default is 100)
# belres: the value of Bellman residual used in the solver (defualt is 1e-3)
solver = ValueIterationSolver(max_iterations=100, belres=1e-3)

# initialize the policy by passing in your problem
policy = ValueIterationPolicy(mdp) 

# solve for an optimal policy
# if verbose=false, the text output will be supressed (false by default)
solve(solver, mdp, policy, verbose=true);

Iteration : 1, residual: 11.161999999999999, iteration run-time: 0.055026756, total run-time: 0.055026756
Iteration : 2, residual: 6.2999734279499995, iteration run-time: 0.044037086, total run-time: 0.09906384200000001
Iteration : 3, residual: 4.535812014371521, iteration run-time: 0.040484037, total run-time: 0.139547879
Iteration : 4, residual: 3.3930953199725944, iteration run-time: 0.04204028, total run-time: 0.18158815900000003
Iteration : 5, residual: 2.5712983606677233, iteration run-time: 0.048566617, total run-time: 0.23015477600000003
Iteration : 6, residual: 1.9231081302159752, iteration run-time: 0.038956967, total run-time: 0.269111743
Iteration : 7, residual: 1.4212677074593327, iteration run-time: 0.040746765, total run-time: 0.30985850800000003
Iteration : 8, residual: 1.101997461260884, iteration run-time: 0.042117354, total run-time: 0.35197586200000003
Iteration : 9, residual: 0.9018018326408601, iteration run-time: 0.046220964, total run-time: 0.39819682600000006
I

Now, we can use the policy along with the ```action(...)``` function to get the optimal action in a given state.

In [36]:
# say we are in state (9,2)
s = GridWorldState(9,2)
a = action(policy, s)

GridWorldAction(:up)

Remeber that the state (9,3) has an immediate reward of +10.0, so the policy we found is moving up as expected!

In [37]:
s = GridWorldState(8,3)
a = action(policy, s)

GridWorldAction(:right)

![description](files/gw.jpg)

## Monte-Carlo Tree Search Solver
Monte-Carlo Tree Search (MCTS) is another MDP solver. It is an online method that looks for the best action from only the current state by building a search tree. A nice overview of MCTS can be found [here](http://pubs.doc.ic.ac.uk/survey-mcts-methods/survey-mcts-methods.pdf). Run the following command to donwload the module:
```julia
Pkg.clone("https://github.com/sisl/MCTS.jl")
```
Let's quickly run through an example of using the solver:

In [38]:
using MCTS

# initialize the problem
mdp = GridWorld()

# initialize the solver
# the hyper parameters in MCTS can be tricky to set properly
# n_iterations: the number of iterations that each search runs for
# depth: the depth of the tree (how far away from the current state the algorithm explores)
# exploration constant: this is how much weight to put into exploratory actions. 
# A good rule of thumb is to set the exploration constant to what you expect the upper bound on your average expected reward to be.
solver = MCTSSolver(n_iterations=100, depth=10, exploration_constant=1.0)

# initialize the policy by passing in your problem and the solver
policy = MCTSPolicy(solver, mdp)

# we don't need to call solver for MCTS

# to get the action:
s = GridWorldState(9,2)
a = action(policy, s)

GridWorldAction(:up)

Let's simulate the policy.

In [40]:
s = GridWorldState(4,1) # this is our starting state
sp = GridWorldState(1,1)

rng = MersenneTwister(9) # initialize a random number generator

rtot = 0.0
disc = 1.0
for i = 1:100
    # get the action from our SARSOP policy
    a = action(policy, s) # the QMDP action function returns the POMDP action not its index like the SARSOP action function
    # compute the reward
    r = reward(mdp, s, a, sp)
    rtot += disc*r
    disc *= discount(mdp)
    
    # transition the system state
    trans_dist = transition(mdp, s, a)
    rand(rng, trans_dist, s)
end
println("Total discounted reward: $rtot")

Total discounted reward: 1.9242953648100016
